# Hospital Financial Reports Explorer (Database Version)

Interactive notebook to explore CMS HCRIS financial statements by hospital (CCN) and fiscal year.

**Data Source:** hospital_worksheets.duckdb (182.5 MB, 25 worksheets)

**Reports Available:**
- Balance Sheet (G000000)
- Income Statement (G300000)
- Statement of Patient Revenues (G200000)
- Statement of Fund Balance Changes (G100000)
- Cost Allocation Summary (B100000)
- Utilization Statistics (S300001)

**Database Info:**
- 229 hospitals (97 NJ, 132 NC)
- Fiscal years 2020-2024
- 2.58 million records

In [ ]:
# Import required libraries
import duckdb
import pandas as pd
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

print("✓ Libraries imported successfully")

In [ ]:
# Connect to database
DB_PATH = 'data/hospital_worksheets.duckdb'

def get_connection():
    """Create a read-only connection to the database"""
    return duckdb.connect(DB_PATH, read_only=True)

# Test connection and show database info
con = get_connection()

# Get table count
tables = con.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'main' AND table_type = 'BASE TABLE'
    ORDER BY table_name
""").df()

worksheet_tables = tables[tables['table_name'].str.startswith('worksheet_')]

print(f"✓ Connected to database: {DB_PATH}")
print(f"✓ Total tables: {len(tables)}")
print(f"✓ Worksheet tables: {len(worksheet_tables)}")
print(f"\nFinancial Statement Worksheets:")
print("  • G000000 - Balance Sheet")
print("  • G100000 - Fund Balance Changes")
print("  • G200000 - Patient Revenues")
print("  • G300000 - Income Statement")
print("\nCost Worksheets:")
print("  • B000001, B000002, B100000 - Cost Allocation")
print("  • A000000 - General Service Costs")

con.close()

In [ ]:
# Get available hospitals and years
con = get_connection()

# Get provider list
providers_df = con.execute("""
    SELECT 
        Provider_Number,
        state_code,
        first_fiscal_year,
        last_fiscal_year,
        fiscal_year_count
    FROM provider_list
    ORDER BY state_code, Provider_Number
""").df()

# Get available fiscal years
years_df = con.execute("""
    SELECT DISTINCT fiscal_year
    FROM all_worksheets
    ORDER BY fiscal_year DESC
""").df()

con.close()

print(f"✓ Found {len(providers_df)} hospitals")
print(f"  • New Jersey (31): {len(providers_df[providers_df['state_code'] == '31'])}")
print(f"  • North Carolina (34): {len(providers_df[providers_df['state_code'] == '34'])}")
print(f"\n✓ Fiscal years available: {', '.join(map(str, years_df['fiscal_year'].tolist()))}")

## Selection Controls

Select a hospital (Provider Number) and fiscal year to view financial reports.

In [ ]:
# Create selection widgets
provider_options = [
    (f"{row['Provider_Number']} (State: {row['state_code']}, Years: {row['fiscal_year_count']})", 
     row['Provider_Number']) 
    for _, row in providers_df.iterrows()
]

provider_dropdown = widgets.Dropdown(
    options=provider_options,
    description='Hospital:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

year_dropdown = widgets.Dropdown(
    options=[(str(year), year) for year in years_df['fiscal_year'].tolist()],
    description='Fiscal Year:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

display(widgets.HBox([provider_dropdown, year_dropdown]))
print("\n👆 Select hospital and fiscal year above")

## Helper Functions

In [ ]:
def format_millions(value):
    """Format value in millions with 2 decimals"""
    if pd.isna(value) or value == 0:
        return 0.00
    return round(value / 1e6, 2)

def style_dataframe(df):
    """Apply styling to dataframe for better readability"""
    return df.style.format({
        col: '{:,.2f}' for col in df.select_dtypes(include=[np.number]).columns
    }).set_properties(**{
        'text-align': 'right'
    }, subset=df.select_dtypes(include=[np.number]).columns).set_properties(**{
        'text-align': 'left'
    }, subset=df.select_dtypes(include=['object']).columns)

print("✓ Helper functions defined")

## 1️⃣ Balance Sheet (G000000)

Complete hospital balance sheet showing assets, liabilities, and net assets/equity.

In [ ]:
def get_balance_sheet(provider_num, year):
    """Get balance sheet for a specific hospital and year"""
    con = get_connection()
    
    df = con.execute("""
        SELECT
            Line,
            line_level1 as Category,
            line_level2 as Subcategory,
            "Column",
            col_level1 as Time_Period,
            Value
        FROM worksheet_g000000
        WHERE Provider_Number = ?
            AND fiscal_year = ?
        ORDER BY Line, "Column"
    """, [provider_num, int(year)]).df()
    
    con.close()
    
    if df.empty:
        print(f"⚠️ No balance sheet data found for Provider {provider_num}, Year {year}")
        return None
    
    # Pivot to show Beginning and Ending balances side by side
    df_pivot = df.pivot_table(
        index=['Line', 'Category', 'Subcategory'],
        columns='Time_Period',
        values='Value',
        aggfunc='first'
    ).reset_index()
    
    # Convert to millions
    for col in df_pivot.select_dtypes(include=[np.number]).columns:
        df_pivot[f'{col} ($M)'] = df_pivot[col].apply(format_millions)
        df_pivot = df_pivot.drop(col, axis=1)
    
    return df_pivot

# Get and display balance sheet
balance_sheet = get_balance_sheet(provider_dropdown.value, year_dropdown.value)

if balance_sheet is not None:
    print(f"\n📊 Balance Sheet (Worksheet G000000)")
    print(f"Provider: {provider_dropdown.value} | Fiscal Year: {year_dropdown.value}")
    print(f"Note: All amounts in millions (USD)")
    print(f"Total line items: {len(balance_sheet):,}\n")
    display(style_dataframe(balance_sheet))

## 2️⃣ Income Statement (G300000)

Statement of revenues and expenses showing operating income and net income.

In [ ]:
def get_income_statement(provider_num, year):
    """Get income statement for a specific hospital and year"""
    con = get_connection()
    
    df = con.execute("""
        SELECT
            Line,
            line_level1 as Account_Type,
            line_level2 as Account_Detail,
            Value
        FROM worksheet_g300000
        WHERE Provider_Number = ?
            AND fiscal_year = ?
        ORDER BY Line
    """, [provider_num, int(year)]).df()
    
    con.close()
    
    if df.empty:
        print(f"⚠️ No income statement data found for Provider {provider_num}, Year {year}")
        return None
    
    # Convert to millions
    df['Value ($M)'] = df['Value'].apply(format_millions)
    df = df.drop('Value', axis=1)
    
    return df

# Get and display income statement
income_stmt = get_income_statement(provider_dropdown.value, year_dropdown.value)

if income_stmt is not None:
    print(f"\n📈 Income Statement (Worksheet G300000)")
    print(f"Provider: {provider_dropdown.value} | Fiscal Year: {year_dropdown.value}")
    print(f"Note: All amounts in millions (USD)")
    print(f"Total line items: {len(income_stmt):,}\n")
    display(style_dataframe(income_stmt))

## 3️⃣ Patient Revenue Statement (G200000)

Detailed breakdown of patient service revenue by payer and service line.

In [ ]:
def get_patient_revenue(provider_num, year):
    """Get patient revenue statement for a specific hospital and year"""
    con = get_connection()
    
    df = con.execute("""
        SELECT
            Line,
            line_level1 as Service_Line,
            line_level2 as Service_Detail,
            "Column",
            col_level1 as Payer_Type,
            Value
        FROM worksheet_g200000
        WHERE Provider_Number = ?
            AND fiscal_year = ?
        ORDER BY Line, "Column"
    """, [provider_num, int(year)]).df()
    
    con.close()
    
    if df.empty:
        print(f"⚠️ No patient revenue data found for Provider {provider_num}, Year {year}")
        return None
    
    # Convert to millions
    df['Value ($M)'] = df['Value'].apply(format_millions)
    df = df.drop('Value', axis=1)
    
    return df

# Get and display patient revenue
patient_revenue = get_patient_revenue(provider_dropdown.value, year_dropdown.value)

if patient_revenue is not None:
    print(f"\n💰 Patient Revenue Statement (Worksheet G200000)")
    print(f"Provider: {provider_dropdown.value} | Fiscal Year: {year_dropdown.value}")
    print(f"Note: All amounts in millions (USD)")
    print(f"Total line items: {len(patient_revenue):,}\n")
    display(style_dataframe(patient_revenue))

## 4️⃣ Fund Balance Changes (G100000)

Statement showing changes in net assets/fund balances during the year.

In [ ]:
def get_fund_balance_changes(provider_num, year):
    """Get fund balance changes for a specific hospital and year"""
    con = get_connection()
    
    df = con.execute("""
        SELECT
            Line,
            line_level1 as Fund_Type,
            line_level2 as Change_Category,
            Value
        FROM worksheet_g100000
        WHERE Provider_Number = ?
            AND fiscal_year = ?
        ORDER BY Line
    """, [provider_num, int(year)]).df()
    
    con.close()
    
    if df.empty:
        print(f"⚠️ No fund balance changes data found for Provider {provider_num}, Year {year}")
        return None
    
    # Convert to millions
    df['Value ($M)'] = df['Value'].apply(format_millions)
    df = df.drop('Value', axis=1)
    
    return df

# Get and display fund balance changes
fund_changes = get_fund_balance_changes(provider_dropdown.value, year_dropdown.value)

if fund_changes is not None:
    print(f"\n💼 Fund Balance Changes (Worksheet G100000)")
    print(f"Provider: {provider_dropdown.value} | Fiscal Year: {year_dropdown.value}")
    print(f"Note: All amounts in millions (USD)")
    print(f"Total line items: {len(fund_changes):,}\n")
    display(style_dataframe(fund_changes))

## 5️⃣ Cost Allocation Summary (B100000)

Final allocated costs by department after stepdown cost allocation.

In [ ]:
def get_cost_allocation(provider_num, year):
    """Get cost allocation summary for a specific hospital and year"""
    con = get_connection()
    
    # Get summary lines (ending in 00) for total allocated costs
    df = con.execute("""
        SELECT
            Line,
            line_level1 as Cost_Center,
            line_level2 as Cost_Detail,
            "Column",
            col_level1 as Allocation_Type,
            Value
        FROM worksheet_b100000
        WHERE Provider_Number = ?
            AND fiscal_year = ?
            AND RIGHT(Line, 2) = '00'
            AND "Column" = '02600'  -- Total column
        ORDER BY Line
    """, [provider_num, int(year)]).df()
    
    con.close()
    
    if df.empty:
        print(f"⚠️ No cost allocation data found for Provider {provider_num}, Year {year}")
        return None
    
    # Convert to millions
    df['Value ($M)'] = df['Value'].apply(format_millions)
    df = df.drop(['Value', 'Column', 'Allocation_Type'], axis=1)
    
    return df

# Get and display cost allocation
cost_allocation = get_cost_allocation(provider_dropdown.value, year_dropdown.value)

if cost_allocation is not None:
    print(f"\n💸 Cost Allocation Summary (Worksheet B100000)")
    print(f"Provider: {provider_dropdown.value} | Fiscal Year: {year_dropdown.value}")
    print(f"Note: All amounts in millions (USD)")
    print(f"Total cost centers: {len(cost_allocation):,}\n")
    display(style_dataframe(cost_allocation))

## 6️⃣ Utilization Statistics (S300001)

Hospital utilization data including admissions, patient days, beds, and occupancy.

In [ ]:
def get_utilization_stats(provider_num, year):
    """Get utilization statistics for a specific hospital and year"""
    con = get_connection()
    
    df = con.execute("""
        SELECT
            Line,
            line_level1 as Statistic_Type,
            line_level2 as Statistic_Detail,
            Value
        FROM worksheet_s300001
        WHERE Provider_Number = ?
            AND fiscal_year = ?
        ORDER BY Line
    """, [provider_num, int(year)]).df()
    
    con.close()
    
    if df.empty:
        print(f"⚠️ No utilization data found for Provider {provider_num}, Year {year}")
        return None
    
    return df

# Get and display utilization statistics
utilization = get_utilization_stats(provider_dropdown.value, year_dropdown.value)

if utilization is not None:
    print(f"\n📊 Utilization Statistics (Worksheet S300001)")
    print(f"Provider: {provider_dropdown.value} | Fiscal Year: {year_dropdown.value}")
    print(f"Total statistics: {len(utilization):,}\n")
    display(style_dataframe(utilization))

## 7️⃣ Key Performance Indicators (KPIs)

Calculate financial ratios and metrics from the data.

In [ ]:
def calculate_kpis(provider_num, year):
    """Calculate key financial metrics"""
    con = get_connection()
    
    kpis = con.execute("""
    WITH balance_sheet AS (
        SELECT
            SUM(CASE WHEN Line BETWEEN '01000' AND '01999' AND "Column" = '00200' THEN Value ELSE 0 END) as current_assets,
            SUM(CASE WHEN Line BETWEEN '10000' AND '10999' AND "Column" = '00200' THEN Value ELSE 0 END) as current_liabilities,
            SUM(CASE WHEN Line BETWEEN '01001' AND '01003' AND "Column" = '00200' THEN Value ELSE 0 END) as cash_and_investments,
            SUM(CASE WHEN Line BETWEEN '10000' AND '19999' AND "Column" = '00200' THEN Value ELSE 0 END) as total_liabilities,
            SUM(CASE WHEN Line BETWEEN '20000' AND '29999' AND "Column" = '00200' THEN Value ELSE 0 END) as net_assets
        FROM worksheet_g000000
        WHERE Provider_Number = ? AND fiscal_year = ?
    ),
    income_stmt AS (
        SELECT
            SUM(CASE WHEN line_level1 LIKE '%Operating Revenue%' THEN Value ELSE 0 END) as operating_revenue,
            SUM(CASE WHEN line_level1 LIKE '%Operating Expense%' THEN Value ELSE 0 END) as operating_expense,
            SUM(CASE WHEN line_level1 LIKE '%Net Income%' THEN Value ELSE 0 END) as net_income
        FROM worksheet_g300000
        WHERE Provider_Number = ? AND fiscal_year = ?
    ),
    utilization AS (
        SELECT
            SUM(CASE WHEN line_level1 LIKE '%Admissions%' THEN Value ELSE 0 END) as admissions,
            SUM(CASE WHEN line_level1 LIKE '%Patient Days%' THEN Value ELSE 0 END) as patient_days
        FROM worksheet_s300001
        WHERE Provider_Number = ? AND fiscal_year = ?
    )
    SELECT
        -- Liquidity
        bs.current_assets / NULLIF(bs.current_liabilities, 0) as current_ratio,
        bs.cash_and_investments / NULLIF((inc.operating_expense / 365), 0) as days_cash_on_hand,
        
        -- Profitability
        ((inc.operating_revenue - inc.operating_expense) / NULLIF(inc.operating_revenue, 0)) * 100 as operating_margin_pct,
        (inc.net_income / NULLIF(inc.operating_revenue, 0)) * 100 as total_margin_pct,
        
        -- Leverage
        bs.total_liabilities / NULLIF(bs.net_assets, 0) as debt_to_equity,
        
        -- Utilization
        util.patient_days / NULLIF(util.admissions, 0) as avg_length_of_stay,
        inc.operating_revenue / NULLIF(util.admissions, 0) as revenue_per_admission
        
    FROM balance_sheet bs
    CROSS JOIN income_stmt inc
    CROSS JOIN utilization util
    """, [provider_num, int(year), provider_num, int(year), provider_num, int(year)]).df()
    
    con.close()
    
    return kpis

# Calculate and display KPIs
kpis = calculate_kpis(provider_dropdown.value, year_dropdown.value)

if not kpis.empty:
    print(f"\n📊 Key Performance Indicators")
    print(f"Provider: {provider_dropdown.value} | Fiscal Year: {year_dropdown.value}\n")
    
    kpis_t = kpis.T
    kpis_t.columns = ['Value']
    kpis_t['Metric'] = kpis_t.index
    kpis_t = kpis_t[['Metric', 'Value']]
    
    display(kpis_t)

## 8️⃣ Export to Excel (Optional)

Export all financial reports to a single Excel file.

In [ ]:
def export_to_excel(provider_num, year):
    """Export all financial reports to Excel file"""
    filename = f"Financial_Reports_Provider_{provider_num}_FY{year}.xlsx"
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        # Balance Sheet
        bs = get_balance_sheet(provider_num, year)
        if bs is not None:
            bs.to_excel(writer, sheet_name='Balance Sheet', index=False)
        
        # Income Statement
        inc = get_income_statement(provider_num, year)
        if inc is not None:
            inc.to_excel(writer, sheet_name='Income Statement', index=False)
        
        # Patient Revenue
        rev = get_patient_revenue(provider_num, year)
        if rev is not None:
            rev.to_excel(writer, sheet_name='Patient Revenue', index=False)
        
        # Fund Balance Changes
        fbc = get_fund_balance_changes(provider_num, year)
        if fbc is not None:
            fbc.to_excel(writer, sheet_name='Fund Balance Changes', index=False)
        
        # Cost Allocation
        costs = get_cost_allocation(provider_num, year)
        if costs is not None:
            costs.to_excel(writer, sheet_name='Cost Allocation', index=False)
        
        # Utilization
        util = get_utilization_stats(provider_num, year)
        if util is not None:
            util.to_excel(writer, sheet_name='Utilization', index=False)
        
        # KPIs
        kpis = calculate_kpis(provider_num, year)
        if not kpis.empty:
            kpis.to_excel(writer, sheet_name='KPIs', index=False)
    
    print(f"✓ Exported to {filename}")
    return filename

# Uncomment to export:
# export_to_excel(provider_dropdown.value, year_dropdown.value)